# ECLAT Algorithm
The ECLAT algorithm stands for _Equivalence Class Clustering and bottom-up Lattice Traversal_. 

It is one of the popular methods of Association Rule mining. 

It is a more efficient and scalable version of the Apriori algorithm.

In [1]:
from collections import defaultdict
from itertools import chain, combinations
from typing import List

In [2]:
txs = [{1, 2, 3, 4}, {1, 2, 4}, {1, 2}, {2, 3, 4}, {2, 3}, {3, 4}, {2, 4}]
txs = [
    ["beer", "wine", "cheese"],
    ["beer", "potato chips"],
    ["eggs", "flower", "butter", "cheese"],
    ["eggs", "flower", "butter", "beer", "potato chips"],
    ["wine", "cheese"],
    ["potato chips"],
    ["eggs", "flower", "butter", "wine", "cheese"],
    ["eggs", "flower", "butter", "beer", "potato chips"],
    ["wine", "beer"],
    ["beer", "potato chips"],
    ["butter", "eggs"],
    ["beer", "potato chips"],
    ["flower", "eggs"],
    ["beer", "potato chips"],
    ["eggs", "flower", "butter", "wine", "cheese"],
    ["beer", "wine", "potato chips", "cheese"],
    ["wine", "cheese"],
    ["beer", "potato chips"],
    ["wine", "cheese"],
    ["beer", "potato chips"],
]

In [3]:
def eclat(txs: List[List[any]], *, minsup=2):
    ids_by_item = defaultdict(set)

    # Invert the mapping, so that the key are items, and values are txids.
    for i, tx in enumerate(txs):
        for item in tx:
            ids_by_item[frozenset([item])].add(i)

    # Exclude items that are below min support.
    for item, ids in ids_by_item.copy().items():
        if len(ids) < minsup:
            ids_by_item.pop(item)

    result = ids_by_item
    while len(ids_by_item) > 0:
        tmp = defaultdict(list)

        keys = combinations(ids_by_item.keys(), r=2)
        for key0, key1 in keys:
            ids0 = ids_by_item.get(key0)
            ids1 = ids_by_item.get(key1)

            ids = ids0 & ids1
            if len(ids) < minsup:
                continue

            tmp[key0 | key1] = ids

        result.update(tmp)
        ids_by_item = tmp

    return result

In [4]:
eclat(txs, minsup=7)

defaultdict(set,
            {frozenset({'beer'}): {0, 1, 3, 7, 8, 9, 11, 13, 15, 17, 19},
             frozenset({'wine'}): {0, 4, 6, 8, 14, 15, 16, 18},
             frozenset({'cheese'}): {0, 2, 4, 6, 14, 15, 16, 18},
             frozenset({'potato chips'}): {1, 3, 5, 7, 9, 11, 13, 15, 17, 19},
             frozenset({'eggs'}): {2, 3, 6, 7, 10, 12, 14},
             frozenset({'beer', 'potato chips'}): {1,
              3,
              7,
              9,
              11,
              13,
              15,
              17,
              19},
             frozenset({'cheese', 'wine'}): {0, 4, 6, 14, 15, 16, 18}})